In [1]:
import pandas as pd
import datetime as dt

In [2]:
#https://goo.gl/xuzLbX
titulos = pd.read_csv('../Datasets/Tesouro Direto (Mensal) - 2005.csv', encoding='iso-8859-1', delimiter =';')

#https://sidra.ibge.gov.br/tabela/1737
ipca = pd.read_csv('../Datasets/IPCA (Mensal) - 1980.csv', delimiter =';')

#https://goo.gl/V5wBxA
selic = pd.read_csv('../Datasets/Selic-DI (Diario) - 2003.csv', encoding='iso-8859-1', delimiter =';')

#http://www.ipeadata.gov.br/Default.aspx
cambio = pd.read_csv('../Datasets/Cambio (Diario) - 1985.csv', delimiter =';')

#https://fred.stlouisfed.org/series/DGS10
treasuries = pd.read_csv('../Datasets/American Treasuries (Diario) - 2003.csv', delimiter =',')

In [3]:
titulos[['dia_base','mes_base','ano_base']] = titulos['Data Base'].str.split('/',expand=True)
titulos = titulos.drop(columns=['Data Base','Preco Compra Manha','Preco Venda Manha','Preco Base Manha','Taxa Venda Manha'])

selic[['dia_base','mes_base','ano_base']] = selic['Data Base'].str.split('/',expand=True)
selic['Taxa DI'] = selic['Media']
selic = selic.drop(columns=['Data Base','Nr Operacoes','Volume','Media','Fator Diario','Minima','Maxima','Moda','Dsv Pdr'])

titulos_selic = pd.merge(titulos, selic, on=['dia_base','mes_base','ano_base'], how='outer')
titulos_selic = titulos_selic.dropna()
titulos_selic.rename(columns={'Tipo Titulo': 'titulo', 'Data Vencimento': 'vencimento', 'Taxa Compra Manha': 'taxa_compra',
                              #'Taxa Venda Manha': 'taxa_venda',
                              'Taxa SELIC': 'selic', 'Taxa DI': 'cdi'}, inplace=True)

cambio[['dia_base','mes_base','ano_base']] = cambio['Data'].str.split('/',expand=True)
cambio = cambio.drop(columns=['Null','Data'])
cambio = cambio.dropna()

titulos_cambio = pd.merge(titulos_selic, cambio, on=['dia_base','mes_base','ano_base'], how='outer')
titulos_cambio = titulos_cambio.dropna()
titulos_cambio.rename(columns={'Taxa_Cambio': 'cambio'}, inplace=True)

treasuries[['ano_base','mes_base','dia_base']] = treasuries['DATE'].str.split('-',expand=True)
treasuries = treasuries.drop(columns=['DATE'])
treasuries = treasuries.dropna()

titulos_treasuries = pd.merge(titulos_cambio, treasuries, on=['dia_base','mes_base','ano_base'], how='outer')
titulos_treasuries = titulos_treasuries.dropna()
titulos_treasuries.rename(columns={'DGS10': 'treasuries'}, inplace=True)

ipca[['mes_base','ano_base']] = ipca['Mes'].str.split(' ',expand=True)
ipca = ipca.drop(columns=['Mes'])
ipca = ipca.rename(columns={'IPCA - Variacao mensal (%)': 'ipca'})

meses = {'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04', 'maio': '05', 'junho': '06',
        'julho': '07', 'agosto': '08', 'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'}
ipca['mes_base'] = ipca['mes_base'].apply(lambda x: meses[x])

titulos_completo = pd.merge(titulos_treasuries, ipca, on=['mes_base','ano_base'], how='outer')
titulos_completo = titulos_completo.dropna()

titulos_completo['data_base'] = titulos_completo[['mes_base', 'dia_base', 'ano_base']].apply(lambda x: '/'.join(x), axis=1)
titulos_completo = titulos_completo.drop(columns=['mes_base', 'dia_base', 'ano_base'])
titulos_completo['data_base'] = pd.to_datetime(titulos_completo.data_base)

titulos_completo['taxa_compra'] = titulos_completo['taxa_compra'].astype(float)
titulos_completo['selic'] = titulos_completo['selic'].astype(float)
titulos_completo['cdi'] = titulos_completo['cdi'].astype(float)
titulos_completo['ipca'] = titulos_completo['ipca'].astype(float)
titulos_completo['cambio'] = titulos_completo['cambio'].astype(float)

titulos_completo = titulos_completo[titulos_completo['treasuries'] != '.']
titulos_completo['treasuries'] = titulos_completo['treasuries'].astype(float)

titulos_completo = titulos_completo.dropna()

titulos_completo = titulos_completo.drop(titulos_completo[(titulos_completo.titulo == 'Tesouro Selic')].index)
titulos_completo['titulo'] = titulos_completo[['titulo', 'vencimento']].apply(lambda x: ' '.join(x), axis=1)
titulos_completo = titulos_completo.drop(columns=['vencimento'])
titulos_completo = titulos_completo.sort_values('data_base', ascending=True)

titulos_completo = titulos_completo.reset_index(drop=True)
titulos_completo.set_index('data_base', inplace=True)
titulos_completo

,titulo,taxa_compra,selic,cdi,cambio,treasuries,ipca
data_base,,,,,,,
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/07/2017,8.26,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/04/2008,8.72,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro Prefixado com Juros Semestrais 01/01/2008,17.14,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/12/2006,8.97,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/03/2011,8.33,17.75,17.76,2.6536,4.24,0.86
...,...,...,...,...,...,...,...
2021-01-14,Tesouro IGPM+ com Juros Semestrais 01/01/2031,3.41,1.90,1.90,5.2611,1.15,0.25
2021-01-14,Tesouro IGPM+ com Juros Semestrais 01/04/2021,0.55,1.90,1.90,5.2611,1.15,0.25
2021-01-14,Tesouro IPCA+ 15/05/2035,3.58,1.90,1.90,5.2611,1.15,0.25


In [4]:
titulos_completo.to_csv('../Datasets/Complete Dataset.csv', index=True)

In [5]:
new_titulos = pd.read_csv('../Datasets/Complete Dataset.csv')
new_titulos.set_index('data_base', inplace=True)
new_titulos

,titulo,taxa_compra,selic,cdi,cambio,treasuries,ipca
data_base,,,,,,,
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/07/2017,8.26,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/04/2008,8.72,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro Prefixado com Juros Semestrais 01/01/2008,17.14,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/12/2006,8.97,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IGPM+ com Juros Semestrais 01/03/2011,8.33,17.75,17.76,2.6536,4.24,0.86
...,...,...,...,...,...,...,...
2021-01-14,Tesouro IGPM+ com Juros Semestrais 01/01/2031,3.41,1.90,1.90,5.2611,1.15,0.25
2021-01-14,Tesouro IGPM+ com Juros Semestrais 01/04/2021,0.55,1.90,1.90,5.2611,1.15,0.25
2021-01-14,Tesouro IPCA+ 15/05/2035,3.58,1.90,1.90,5.2611,1.15,0.25


In [6]:
IPCA = new_titulos[new_titulos['titulo'].str.contains("Tesouro IPCA+")]
IPCA.head(20)

,titulo,taxa_compra,selic,cdi,cambio,treasuries,ipca
data_base,,,,,,,
2004-12-31,Tesouro IPCA+ com Juros Semestrais 15/05/2009,8.78,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IPCA+ com Juros Semestrais 15/08/2006,8.71,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IPCA+ com Juros Semestrais 15/05/2015,8.76,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IPCA+ com Juros Semestrais 15/08/2024,8.83,17.75,17.76,2.6536,4.24,0.86
2004-12-31,Tesouro IPCA+ com Juros Semestrais 15/05/2045,9.05,17.75,17.76,2.6536,4.24,0.86
2005-01-03,Tesouro IPCA+ com Juros Semestrais 15/08/2006,8.71,17.74,17.76,2.6674,4.23,0.58
2005-01-03,Tesouro IPCA+ com Juros Semestrais 15/05/2009,8.78,17.74,17.76,2.6674,4.23,0.58
2005-01-03,Tesouro IPCA+ com Juros Semestrais 15/05/2015,8.76,17.74,17.76,2.6674,4.23,0.58
2005-01-03,Tesouro IPCA+ com Juros Semestrais 15/08/2024,8.83,17.74,17.76,2.6674,4.23,0.58


In [7]:
IPCA.titulo.unique()

array(['Tesouro IPCA+ com Juros Semestrais 15/05/2009',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2006',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2015',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2024',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2045',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2007',
       'Tesouro IPCA+ 15/05/2015', 'Tesouro IPCA+ 15/08/2024',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2008',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2010',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2011',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2035',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2012',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2017',
       'Tesouro IPCA+ com Juros Semestrais 15/05/2013',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2020',
       'Tesouro IPCA+ 15/05/2035',
       'Tesouro IPCA+ com Juros Semestrais 15/08/2050',
       'Tesouro IPCA+ 15/05/2019',
       'Tesouro IPCA+ com J

In [12]:
IPCA_2024 = IPCA[IPCA['titulo']=='Tesouro IPCA+ com Juros Semestrais 15/08/2024']
IPCA_2024 = IPCA_2024.drop(columns=['titulo'])
IPCA_2024

,taxa_compra,selic,cdi,cambio,treasuries,ipca
data_base,,,,,,
2004-12-31,8.83,17.75,17.76,2.6536,4.24,0.86
2005-01-03,8.83,17.74,17.76,2.6674,4.23,0.58
2005-01-04,8.83,17.74,17.72,2.6879,4.29,0.58
2005-01-05,8.83,17.74,17.72,2.7088,4.29,0.58
2005-01-06,8.83,17.74,17.72,2.7199,4.29,0.58
...,...,...,...,...,...,...
2021-01-08,1.66,1.90,1.90,5.3677,1.13,0.25
2021-01-11,1.77,1.90,1.90,5.4960,1.15,0.25
2021-01-12,1.92,1.90,1.90,5.4631,1.15,0.25
